## Logic Based FizzBuzz Function [Software 1.0]

In [1]:
import pandas as pd

def fizzbuzz(n):
    
    # Logic Explanation
    if n % 3 == 0 and n % 5 == 0:
        return 'fizzbuzz'
    elif n % 3 == 0:
        return 'fizz'
    elif n % 5 == 0:
        return 'buzz'
    else:
        return 'other'

## Create Training and Testing Datasets in CSV Format

In [2]:
def createInputCSV(start,end,filename):
    
    # Why list in Python?
    inputData   = []
    outputData  = []
    
    # Why do we need training Data?
    for i in range(start,end):
        inputData.append(i)
        outputData.append(fizzbuzz(i))
    
    # Why Dataframe?
    # Dataframes are pandas objects, which themselves are enhanced version of 
    # numpy structured arrays
    # DataFrames are inherently multidimensional arrays with 
    # attached row and column labels, and added support for heterogeneous 
    # types and missing data.
    dataset = {}
    dataset["input"]  = inputData
    dataset["label"] = outputData
    
    # Writing to csv
    pd.DataFrame(dataset).to_csv(filename)
    
    print(filename, "Created!")

## Processing Input and Label Data

In [3]:
def processData(dataset):
    """Processes input data and label.
    
    Parameters
    ---------
        dataset : pandas dataframe
            Dataframe containing data and labels

    Returns
    -------
        processedData : list
        processedLabel :  list
    """
    # Why do we have to process?
    # Preprocess input data to make it compatible for Neural Network
    data   = dataset['input'].values
    labels = dataset['label'].values
    
    processedData  = encodeData(data)
    processedLabel = encodeLabel(labels)
    
    return processedData, processedLabel

In [4]:
def encodeData(data):
    """Converts a number to its binary reporesentaiton
    """
    processedData = []
    
    for dataInstance in data:
        
        # Why do we have number 10?
        processedData.append([dataInstance >> d & 1 for d in range(10)])
    
    return np.array(processedData)

In [5]:
from keras.utils import np_utils
import os

# TF_CPP_MIN_LOG_LEVEL : Tensorflow environment variable
# Default: to 0 (all logs shown), 
# 1 to filter out INFO logs, 
# 2 to additionally filter out WARNING logs, 
# 3 to additionally filter out ERROR logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def encodeLabel(labels):
    """
    """
    processedLabel = []
    
    for labelInstance in labels:
        if(labelInstance == "fizzbuzz"):
            # Fizzbuzz
            processedLabel.append([3])
        elif(labelInstance == "fizz"):
            # Fizz
            processedLabel.append([1])
        elif(labelInstance == "buzz"):
            # Buzz
            processedLabel.append([2])
        else:
            # Other
            processedLabel.append([0])
    
    return np_utils.to_categorical(np.array(processedLabel),4)

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Model Definition

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, TensorBoard

import numpy as np

def get_model():
    """Returns Sequential NN model

    Parameters
    ----------

    Returns
    -------
        model: keras Sequential object
    """

    # MODEL PARAMETERS
    # Hidden Layer
    input_size = 10  # 2^10 > 1000
    drop_out = 0.4  # adjusting for overfitting
    first_dense_layer_nodes = 512  # first hidden layer
    # Note : hidden layer units shouldn't be too absurd,
    # not to large (overfitting), not too small
    second_dense_layer_nodes = 4  # output layer, no of classes of classification problem


    # Why do we need a model?
    # 'Model' is a data structure used by keras, a way to
    # organize the layers.

    # Why use Dense layer and then activation?
    # Dense layer means fully connected layer.

    # Why use sequential model with layers?
    model = Sequential()

    # First Layer(Input Layer): Dimensionality 900*10
    # 900 numbers (101-1000), 10 bits each
    # Hidden Layer:
    # Weights: 10 * 256 (10 bits, 256 units in the first layer)
    # Dimensions from output of first layer = 900 * 256
    model.add(Dense(first_dense_layer_nodes, input_dim=input_size))
    model.add(Activation('relu'))
    model.add(Dropout(drop_out))
    
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(drop_out))

    # Activation : Functions sigmoid/relu
    # Why Relu :relu is also used in regularization
    # Why not sigmoid?


    # Why dropout?
    # Dropout is a technique where randomly selected neurons are ignored during training.
    # The effect is that the network becomes less sensitive to the specific weights of neurons.
    # This in turn results in a network that is capable of better generalization and is less likely to
    # overfit the training data.
#     model.add(Dropout(drop_out))
 
    # Output Layer
    model.add(Dense(second_dense_layer_nodes))
    model.add(Activation('softmax'))
    # Why Softmax?

    model.summary()

    # Experiment with optimizers:
    # SGD, adagrad, adam etc.

    # Loss function
    # Cross Entropy: KL Divergence
    # Why use categorical_crossentropy?
    # Neural Network Compilation Step
    model.compile(
        optimizer='adadelta',
        loss='categorical_crossentropy',
        metrics=['accuracy'])

    return model


# <font color='blue'>Creating Training and Testing Datafiles</font>

In [7]:
# Create datafiles
createInputCSV(101,1001,'training.csv')
createInputCSV(1,101,'testing.csv')

training.csv Created!
testing.csv Created!


# <font color='blue'>Creating Model</font>

In [8]:
model = get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               5632      
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 1028      
__________

# <font color = blue>Run Model</font>

In [9]:
validation_data_split = 0.2 # trying to tackle the problem of overfitting
# difference between validation to testing
# after every epoch we validate
# validation loss and training loss
# validation avoids over fitting

# cros validation: validation data keeps rotating


num_epochs = 1000 # experiment on this
model_batch_size = 128
# minibatch? : between the 2 approaches:
# fitting whole data into the model, 

tb_batch_size = 32
early_patience = 100

tensorboard_cb   = TensorBoard(log_dir='logs', batch_size= tb_batch_size, write_graph= True)
earlystopping_cb = EarlyStopping(monitor='val_loss', verbose=1, patience=early_patience, mode='min')

# Read Dataset
dataset = pd.read_csv('training.csv')

# Process Dataset
processedData, processedLabel = processData(dataset)

# Training Loop
history = model.fit(processedData
                    , processedLabel
                    , validation_split=validation_data_split
                    , epochs=num_epochs
                    , batch_size=model_batch_size
                    , callbacks = [tensorboard_cb,earlystopping_cb]
                   )

Train on 720 samples, validate on 180 samples
Epoch 1/1000
720/720 [==============================] - 0s 332us/step - loss: 1.2685 - acc: 0.4403 - val_loss: 1.1550 - val_acc: 0.5333
Epoch 2/1000
720/720 [==============================] - 0s 26us/step - loss: 1.1799 - acc: 0.5333 - val_loss: 1.1492 - val_acc: 0.5333
Epoch 3/1000
720/720 [==============================] - 0s 25us/step - loss: 1.1667 - acc: 0.5292 - val_loss: 1.1472 - val_acc: 0.5333
Epoch 4/1000
720/720 [==============================] - 0s 26us/step - loss: 1.1623 - acc: 0.5319 - val_loss: 1.1543 - val_acc: 0.5333
Epoch 5/1000
720/720 [==============================] - 0s 26us/step - loss: 1.1492 - acc: 0.5333 - val_loss: 1.1442 - val_acc: 0.5333
Epoch 6/1000
720/720 [==============================] - 0s 26us/step - loss: 1.1497 - acc: 0.5333 - val_loss: 1.1461 - val_acc: 0.5333
Epoch 7/1000
720/720 [==============================] - 0s 26us/step - loss: 1.1509 - acc: 0.5347 - val_loss: 1.1621 - val_acc: 0.5333
Epoch 8/

Epoch 61/1000
720/720 [==============================] - 0s 26us/step - loss: 0.9924 - acc: 0.5514 - val_loss: 1.0845 - val_acc: 0.5333
Epoch 62/1000
720/720 [==============================] - 0s 26us/step - loss: 1.0020 - acc: 0.5611 - val_loss: 1.0962 - val_acc: 0.5278
Epoch 63/1000
720/720 [==============================] - 0s 26us/step - loss: 0.9775 - acc: 0.5611 - val_loss: 1.1257 - val_acc: 0.5111
Epoch 64/1000
720/720 [==============================] - 0s 25us/step - loss: 0.9615 - acc: 0.5681 - val_loss: 1.0830 - val_acc: 0.5333
Epoch 65/1000
720/720 [==============================] - 0s 26us/step - loss: 0.9782 - acc: 0.5556 - val_loss: 1.0689 - val_acc: 0.5333
Epoch 66/1000
720/720 [==============================] - 0s 26us/step - loss: 0.9801 - acc: 0.5611 - val_loss: 1.0782 - val_acc: 0.5333
Epoch 67/1000
720/720 [==============================] - 0s 27us/step - loss: 0.9573 - acc: 0.5722 - val_loss: 1.0640 - val_acc: 0.5278
Epoch 68/1000
720/720 [=========================

720/720 [==============================] - 0s 27us/step - loss: 0.5778 - acc: 0.7667 - val_loss: 0.6323 - val_acc: 0.8500
Epoch 122/1000
720/720 [==============================] - 0s 27us/step - loss: 0.5541 - acc: 0.8028 - val_loss: 0.6543 - val_acc: 0.7833
Epoch 123/1000
720/720 [==============================] - 0s 28us/step - loss: 0.5898 - acc: 0.7792 - val_loss: 0.6612 - val_acc: 0.7667
Epoch 124/1000
720/720 [==============================] - 0s 28us/step - loss: 0.5514 - acc: 0.8000 - val_loss: 0.6359 - val_acc: 0.7667
Epoch 125/1000
720/720 [==============================] - 0s 27us/step - loss: 0.5675 - acc: 0.7778 - val_loss: 0.6378 - val_acc: 0.7833
Epoch 126/1000
720/720 [==============================] - 0s 28us/step - loss: 0.5080 - acc: 0.8056 - val_loss: 0.6057 - val_acc: 0.7722
Epoch 127/1000
720/720 [==============================] - 0s 27us/step - loss: 0.4955 - acc: 0.8306 - val_loss: 0.6058 - val_acc: 0.8333
Epoch 128/1000
720/720 [==============================] 

Epoch 181/1000
720/720 [==============================] - 0s 28us/step - loss: 0.2927 - acc: 0.8861 - val_loss: 0.3085 - val_acc: 0.9167
Epoch 182/1000
720/720 [==============================] - 0s 27us/step - loss: 0.2763 - acc: 0.9083 - val_loss: 0.3163 - val_acc: 0.9278
Epoch 183/1000
720/720 [==============================] - 0s 28us/step - loss: 0.2993 - acc: 0.8944 - val_loss: 0.3213 - val_acc: 0.9000
Epoch 184/1000
720/720 [==============================] - 0s 31us/step - loss: 0.2633 - acc: 0.9125 - val_loss: 0.3085 - val_acc: 0.9389
Epoch 185/1000
720/720 [==============================] - 0s 31us/step - loss: 0.2711 - acc: 0.9083 - val_loss: 0.2812 - val_acc: 0.9444
Epoch 186/1000
720/720 [==============================] - 0s 29us/step - loss: 0.2675 - acc: 0.9042 - val_loss: 0.2929 - val_acc: 0.9444
Epoch 187/1000
720/720 [==============================] - 0s 28us/step - loss: 0.2837 - acc: 0.9056 - val_loss: 0.3280 - val_acc: 0.9333
Epoch 188/1000
720/720 [=================

Epoch 241/1000
720/720 [==============================] - 0s 28us/step - loss: 0.1883 - acc: 0.9347 - val_loss: 0.2415 - val_acc: 0.9389
Epoch 242/1000
720/720 [==============================] - 0s 27us/step - loss: 0.1761 - acc: 0.9278 - val_loss: 0.2294 - val_acc: 0.9556
Epoch 243/1000
720/720 [==============================] - 0s 25us/step - loss: 0.1673 - acc: 0.9472 - val_loss: 0.2281 - val_acc: 0.9500
Epoch 244/1000
720/720 [==============================] - 0s 24us/step - loss: 0.2032 - acc: 0.9181 - val_loss: 0.2170 - val_acc: 0.9500
Epoch 245/1000
720/720 [==============================] - 0s 26us/step - loss: 0.1784 - acc: 0.9319 - val_loss: 0.2129 - val_acc: 0.9500
Epoch 246/1000
720/720 [==============================] - 0s 25us/step - loss: 0.1531 - acc: 0.9514 - val_loss: 0.2239 - val_acc: 0.9500
Epoch 247/1000
720/720 [==============================] - 0s 24us/step - loss: 0.1671 - acc: 0.9375 - val_loss: 0.2195 - val_acc: 0.9500
Epoch 248/1000
720/720 [=================

Epoch 301/1000
720/720 [==============================] - 0s 24us/step - loss: 0.1280 - acc: 0.9514 - val_loss: 0.2182 - val_acc: 0.9500
Epoch 302/1000
720/720 [==============================] - 0s 24us/step - loss: 0.1284 - acc: 0.9542 - val_loss: 0.2145 - val_acc: 0.9500
Epoch 303/1000
720/720 [==============================] - 0s 24us/step - loss: 0.1179 - acc: 0.9625 - val_loss: 0.2028 - val_acc: 0.9444
Epoch 304/1000
720/720 [==============================] - 0s 24us/step - loss: 0.1371 - acc: 0.9542 - val_loss: 0.2247 - val_acc: 0.9500
Epoch 305/1000
720/720 [==============================] - 0s 26us/step - loss: 0.1327 - acc: 0.9583 - val_loss: 0.2148 - val_acc: 0.9500
Epoch 306/1000
720/720 [==============================] - 0s 24us/step - loss: 0.1333 - acc: 0.9528 - val_loss: 0.2079 - val_acc: 0.9500
Epoch 307/1000
720/720 [==============================] - 0s 24us/step - loss: 0.1403 - acc: 0.9556 - val_loss: 0.2123 - val_acc: 0.9556
Epoch 308/1000
720/720 [=================

Epoch 361/1000
720/720 [==============================] - 0s 23us/step - loss: 0.0993 - acc: 0.9694 - val_loss: 0.2375 - val_acc: 0.9500
Epoch 362/1000
720/720 [==============================] - 0s 24us/step - loss: 0.0981 - acc: 0.9681 - val_loss: 0.2163 - val_acc: 0.9500
Epoch 363/1000
720/720 [==============================] - 0s 24us/step - loss: 0.1066 - acc: 0.9667 - val_loss: 0.2120 - val_acc: 0.9500
Epoch 364/1000
720/720 [==============================] - 0s 30us/step - loss: 0.0968 - acc: 0.9681 - val_loss: 0.2051 - val_acc: 0.9500
Epoch 365/1000
720/720 [==============================] - 0s 26us/step - loss: 0.0833 - acc: 0.9722 - val_loss: 0.1976 - val_acc: 0.9500
Epoch 366/1000
720/720 [==============================] - 0s 24us/step - loss: 0.0895 - acc: 0.9722 - val_loss: 0.1948 - val_acc: 0.9556
Epoch 367/1000
720/720 [==============================] - 0s 24us/step - loss: 0.0895 - acc: 0.9722 - val_loss: 0.2064 - val_acc: 0.9500
Epoch 368/1000
720/720 [=================

Epoch 421/1000
720/720 [==============================] - 0s 25us/step - loss: 0.0907 - acc: 0.9625 - val_loss: 0.1990 - val_acc: 0.9611
Epoch 422/1000
720/720 [==============================] - 0s 26us/step - loss: 0.0668 - acc: 0.9750 - val_loss: 0.1978 - val_acc: 0.9556
Epoch 423/1000
720/720 [==============================] - 0s 23us/step - loss: 0.0644 - acc: 0.9736 - val_loss: 0.1959 - val_acc: 0.9556
Epoch 424/1000
720/720 [==============================] - 0s 24us/step - loss: 0.0760 - acc: 0.9792 - val_loss: 0.2079 - val_acc: 0.9556
Epoch 425/1000
720/720 [==============================] - 0s 26us/step - loss: 0.0684 - acc: 0.9778 - val_loss: 0.2019 - val_acc: 0.9500
Epoch 426/1000
720/720 [==============================] - 0s 23us/step - loss: 0.0948 - acc: 0.9681 - val_loss: 0.2364 - val_acc: 0.9500
Epoch 427/1000
720/720 [==============================] - 0s 23us/step - loss: 0.0746 - acc: 0.9722 - val_loss: 0.2092 - val_acc: 0.9500
Epoch 428/1000
720/720 [=================

Epoch 481/1000
720/720 [==============================] - 0s 24us/step - loss: 0.0627 - acc: 0.9806 - val_loss: 0.2008 - val_acc: 0.9500
Epoch 482/1000
720/720 [==============================] - 0s 25us/step - loss: 0.0825 - acc: 0.9764 - val_loss: 0.2014 - val_acc: 0.9556
Epoch 483/1000
720/720 [==============================] - 0s 25us/step - loss: 0.0611 - acc: 0.9806 - val_loss: 0.2023 - val_acc: 0.9500
Epoch 484/1000
720/720 [==============================] - 0s 24us/step - loss: 0.0759 - acc: 0.9750 - val_loss: 0.2047 - val_acc: 0.9556
Epoch 485/1000
720/720 [==============================] - 0s 24us/step - loss: 0.0542 - acc: 0.9875 - val_loss: 0.2020 - val_acc: 0.9556
Epoch 486/1000
720/720 [==============================] - 0s 24us/step - loss: 0.0680 - acc: 0.9764 - val_loss: 0.2031 - val_acc: 0.9611
Epoch 487/1000
720/720 [==============================] - 0s 24us/step - loss: 0.0652 - acc: 0.9833 - val_loss: 0.2388 - val_acc: 0.9444
Epoch 488/1000
720/720 [=================

# <font color = blue>Training and Validation Graphs</font>

In [10]:
df = pd.DataFrame(history.history)
# print(df)
df.plot(subplots=True, grid=True, figsize=(10,15))

array([<matplotlib.axes._subplots.AxesSubplot object at 0x128122908>,
      dtype=object)

# <font color = blue>Testing Accuracy [Software 2.0]</font>

In [11]:
def decodeLabel(encodedLabel):
    if encodedLabel == 0:
        return "other"
    elif encodedLabel == 1:
        return "fizz"
    elif encodedLabel == 2:
        return "buzz"
    elif encodedLabel == 3:
        return "fizzbuzz"

In [12]:
wrong   = 0
right   = 0

testData = pd.read_csv('testing.csv')

processedTestData  = encodeData(testData['input'].values)
processedTestLabel = encodeLabel(testData['label'].values)
predictedTestLabel = []

for i,j in zip(processedTestData,processedTestLabel):
    y = model.predict(np.array(i).reshape(-1,10))
    predictedTestLabel.append(decodeLabel(y.argmax()))
    
    if j.argmax() == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))

print("Testing Accuracy: " + str(right/(right+wrong)*100))

output = {}
output["input"] = testData['input']
output["label"] = testData['label']
output["predicted_label"] = predictedTestLabel

opdf = pd.DataFrame(output)
opdf.to_csv('output.csv')

Errors: 1  Correct :99
Testing Accuracy: 99.0
